**Mohamad Chehade**

Import the dataset with the players:

In [21]:
import pandas as pd

df = pd.read_excel (r'Players - New.xlsx')

Visualize the dataset:

In [22]:
df.head()

,Overall Rank,FW Rank,MID Rank,DEF Rank,GK Rank,Player,Team,Pos,Price,TSB%,Pts,MIN
0,1,NaN,1.0,NaN,NaN,Mohamed Salah,LIV,M,13.0,61.5,272.30,2800
1,2,NaN,2.0,NaN,NaN,Son Heung-Min,TOT,M,12.0,24.9,245.70,2900
2,3,NaN,3.0,NaN,NaN,Kevin De Bruyne,MCI,M,12.0,14.9,230.40,2600
3,4,NaN,NaN,1.0,NaN,Joao Cancelo,MCI,D,7.0,45.5,227.25,3000
4,5,NaN,NaN,2.0,NaN,Trent Alexander-Arnold,LIV,D,7.5,59.6,225.50,2800


sort variables by team:

In [23]:
df=df.sort_values(by=['Team']) ; 

Check new dataframe:

In [24]:
df.head()

,Overall Rank,FW Rank,MID Rank,DEF Rank,GK Rank,Player,Team,Pos,Price,TSB%,Pts,MIN
241,242,36.0,NaN,NaN,NaN,Eddie Nketiah,ARS,F,7.0,0.9,70.50,1200
96,97,NaN,40.0,NaN,NaN,Gabriel Martinelli,ARS,M,6.0,21.3,106.70,2000
46,47,NaN,NaN,13.0,NaN,Gabriel,ARS,D,5.0,8.7,135.75,2700
196,197,NaN,76.0,NaN,NaN,Fabio Vieira,ARS,M,6.0,0.2,83.85,1600
40,41,NaN,NaN,NaN,5.0,Aaron Ramsdale,ARS,G,5.0,14.3,138.90,3420


In [25]:
df.index=range(1,len(df)+1)
df["index"]=df.index

In [26]:
df.head()

,Overall Rank,FW Rank,MID Rank,DEF Rank,GK Rank,Player,Team,Pos,Price,TSB%,Pts,MIN,index
1,242,36.0,NaN,NaN,NaN,Eddie Nketiah,ARS,F,7.0,0.9,70.50,1200,1
2,97,NaN,40.0,NaN,NaN,Gabriel Martinelli,ARS,M,6.0,21.3,106.70,2000,2
3,47,NaN,NaN,13.0,NaN,Gabriel,ARS,D,5.0,8.7,135.75,2700,3
4,197,NaN,76.0,NaN,NaN,Fabio Vieira,ARS,M,6.0,0.2,83.85,1600,4
5,41,NaN,NaN,NaN,5.0,Aaron Ramsdale,ARS,G,5.0,14.3,138.90,3420,5


Create dictionary that maps:
Position -> set of indices 

In [27]:
Dict = {"G":[],"D":[],"M":[],"F":[]}
x=0
for i in range(1,len(df)+1):
  Dict[df.loc[i,"Pos"]].append(df.loc[i,"index"]-1)

Create list of stop values for each team: 

In [28]:
Teams = []
for i in range(1,len(df)):
    if df.loc[i,"Team"]!=df.loc[i+1,"Team"]:
      Teams.append(i)
Teams.append(len(df))

install library

In [29]:
!python -m pip install --upgrade --user -q ortools

You should consider upgrading via the 'C:\Users\HP\Anaconda\python.exe -m pip install --upgrade pip' command.


Import the library

In [30]:
from ortools.linear_solver import pywraplp

Initialize solver:

In [31]:
solver = pywraplp.Solver('Maximize Points', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

Create the player boolean variables:

In [32]:
Variables = [1]*len(df)
for i in range(len(df)):
  Variables[i] = solver.IntVar(0,1,df.loc[i+1,"Player"])

Create slack variables 

In [33]:
z1 = solver.IntVar(0,1,"z1")
z2 = solver.IntVar(0,1,"z2")

Create slack constants

In [34]:
M = 100 
epsilon = 1e-3

Create objective function:
maximize player_boolean*player_points

In [35]:
Objective = 0
for i in range(len(df)):
  Objective = Objective + Variables[i]*df.loc[i+1,"Pts"]
solver.Maximize(Objective)

Add team constraint: player_boolean_team <=3

In [36]:
for i in range (1,len(Teams)):
  Team_constraint = 0
  if i==0: 
    Team_constraint +=Variables[0:Teams[i]]
    solver.Add(Team_constraint<=3)
  else:
    for j in range(Teams[i-1],Teams[i]):
          Team_constraint += Variables[j]
    solver.Add(Team_constraint<=3)

Add number of players constraint:

In [37]:
Number = 0
for i in range(len(df)):
  Number = Number + Variables[i]
solver.Add(Number==11)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000223C7670F30> >

Add positions constraint 

In [38]:
Goalkeeper = 0
for i in range(len(Dict["G"])):
  Goalkeeper = Goalkeeper + Variables[Dict["G"][i]]
solver.Add(Goalkeeper == 1)

Defender = 0
for i in range(len(Dict["D"])):
  Defender = Defender + Variables[Dict["D"][i]]
solver.Add(Defender >= 3)
solver.Add(Defender <= 5)

Midfielder = 0
for i in range(len(Dict["M"])):
  Midfielder = Midfielder + Variables[Dict["M"][i]]
solver.Add(Midfielder >= 2)
solver.Add(Midfielder <= 5)

Forward = 0
for i in range(len(Dict["F"])):
  Forward  = Forward  + Variables[Dict["F"][i]]
solver.Add(Forward  >= 1)
solver.Add(Forward  <= 3)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000223C7705690> >

Price Constraint: player_boolean*player_price <=83.5

In [39]:
Price = 0
for i in range(len(df)):
  Price = Price + Variables[i]*df.loc[i+1,"Price"]
solver.Add(Price<=83.5)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000223C799F510> >

Case of 5 defenders (Budget: 82.5)

In [40]:
solver.Add(5 - Defender <= M*z1) 
solver.Add(Defender - 5 <= M*(1-z1) - epsilon) 
solver.Add(Price - 82.5 <= M*z1)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000223C767DC60> >

Case of 4 defenders (Budget: 83)

In [41]:
solver.Add(4 - Defender <= M*z2)
solver.Add(Defender - 4 <= M*(1-z2) - epsilon) 
solver.Add(Price - 83 <= M*z2)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x00000223C767DF30> >

In [42]:
status = solver.Solve()

In [43]:
# If an optimal solution has been found, print results
if status == pywraplp.Solver.OPTIMAL:
  print('================= Solution =================')
  print("Number of Points = " + str(Objective.solution_value()+265)+"\n")
  for i in range(1,len(df)+1):
        if Variables[i-1].solution_value()==1:
          print(df.loc[i,"Player"] + " (" + df.loc[i,"Team"] +")" + " = " + str(df.loc[i,"Pts"])+"\n")
  print("Price = " + str(Price.solution_value()) + "m"+"\n")
  print("Defenders = " + str(Defender.solution_value())+"\n")
  print("Midfielders = " + str(Midfielder.solution_value())+"\n")
  print("Forwards = " + str(Forward.solution_value())+"\n")
else:
  print('The solver could not find an optimal solution.')

================= Solution =================
Number of Points = 2445.8999999999996

Tyrone Mings (AVL) = 123.5

Reece James (CHE) = 188.25

Wilfried Zaha (CRY) = 188.5

Aleksandar Mitrovic (FUL) = 170.15

Trent Alexander-Arnold (LIV) = 225.5

Alisson (LIV) = 177.9

Mohamed Salah (LIV) = 272.3

Phil Foden (MCI) = 192.35

Joao Cancelo (MCI) = 227.25

Son Heung-Min (TOT) = 245.7

Ivan Perisic (TOT) = 169.5

Price = 82.5m

Defenders = 5.0

Midfielders = 4.0

Forwards = 1.0

